# How to use Kompy
# --------------

to install Kompy, you can use pip:

```bash
pip install kompy
```

You need to import the library:

```python
import kompy as kp
```

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

## Set up the KomootConnector

The first component is the KomootConnector, this class can be used to download or upload tour(s) and get informations
 about the users.
 
Unfortunately, Komoot doesn't provide an oauth2 authentication method for those that are not partners, as a 
consequence, I could implement only a simple http authentication method. 

In [ ]:
from kompy import KomootConnector

In [ ]:
connector = KomootConnector(
    password=os.environ['KOMOOT_PSW'],
    email=os.environ['KOMOOT_email'],
)

We can then easily verify that, given that you set the correct env variables or you passed the correct credentials, 
we correctly logged in

In [ ]:
print(f'Authentication:\n{connector.authentication}')

## Get the user's tours

We can now get the user's tours, we can do it in two ways:
- get the tours of the current user
- get the tours of another user

For both those usecases we'll use the `get_tours` method. This method accepts many parameters, but the most important
 is the `user_identifier`. This parameter is the identifier of the user that we want to get the tours from. If we want
  to get the tours of the current user, we can simply pass `None` as the `user_identifier` parameter.


In [ ]:
tours_list = connector.get_tours(user_identifier=None)

Depending on how many tours you have, this method can take a while to complete, so be patient.

The returned object is a list of `Tour` objects, each of those objects contains all the information about the tour.

In [ ]:
print(f'Tour object: {tours_list[0]}')

Not all the fields of the `Tour` object are populated, this is because the Komoot API doesn't return all the fields, 
for example, to fetch all the coordinates of the tour, we need to call the `get_tour_coordinates` method.

In [ ]:
t1 = tours_list[0]
print(f'Coordinates not yet fetched: {t1.coordinates is []}')
t1.get_coordinates(connector.authentication)
print(f'Coordinates fetched: {t1.coordinates is not []}')

The `Coordinate` object is a simple object that contains the latitude and longitude of the point, the altitude and 
the timestamp.

In [ ]:
print(f'Latitude of the first coordinate point: {t1.coordinates[0].lat}')

## Get a single tour

For some reasons, you might want to fetch a single tour object, be it in the kompy format or in GPX/FIT format. you 
can do so by using the `get_tour` method.

In [ ]:
tour = connector.get_tour_by_id(tour_identifier=t1.id)

The returned object is the same as the ones fetched by the get_tours method. You can also get the tour in GPX or FIT

In [ ]:
gpx_tour = connector.get_tour_by_id(tour_identifier=t1.id, object_type='gpx')
print(f'Format of tour: {type(gpx_tour)}')

## Upload a tour

It is also possible to upload a tour, in gpx or fit format. To do so, you can use the `upload_tour` method.

In [ ]:
success_bool = connector.upload_tour(
    tour_object=gpx_tour,
    activity_type='hiking',
    tour_name='test',
    time_in_motion=42,
)

In [ ]:
print(f'Upload success: {success_bool}')